In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [2]:
# Plot candle price
pair ="GBP_JPY"
granularity = "H1"
ma_list = [9,21]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)



In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_9,MA_21
0,2023-01-18T10:00:00.000000000Z,159.920,160.093,159.326,159.527,160.406556,158.771762
1,2023-01-18T11:00:00.000000000Z,159.524,160.047,159.420,159.589,160.337333,158.862762
2,2023-01-18T12:00:00.000000000Z,159.589,159.759,159.400,159.664,160.200444,158.972762
3,2023-01-18T13:00:00.000000000Z,159.664,159.768,158.811,158.818,159.922000,159.050762
4,2023-01-18T14:00:00.000000000Z,158.818,159.119,158.602,159.030,159.739111,159.135238


In [6]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 #Buy
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 #Sell
    return 0

In [7]:
df_ma['DIFF'] = df_ma.MA_9 - df_ma.MA_21
df_ma['DIFF_PREV']= df_ma.DIFF.shift(1)
df_ma['IS_TRADE']= df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()


In [8]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_9,MA_21,DIFF,DIFF_PREV,IS_TRADE
7,2023-01-18T17:00:00.000000000Z,158.878,159.164,158.802,159.034,159.284333,159.354714,-0.070381,0.113302,-1
29,2023-01-19T15:00:00.000000000Z,158.552,158.815,158.500,158.764,158.489000,158.472952,0.016048,-0.102921,1
93,2023-01-24T07:00:00.000000000Z,161.302,161.333,160.945,161.085,161.361556,161.427333,-0.065778,0.010921,-1
112,2023-01-25T02:00:00.000000000Z,160.574,160.734,160.456,160.616,160.556778,160.549095,0.007683,-0.059603,1
121,2023-01-25T11:00:00.000000000Z,159.904,159.926,159.512,159.606,160.399222,160.445810,-0.046587,0.010349,-1


In [9]:
df_trades['DELTA']= (df_trades.mid_c.diff()/i_pair.pipLocation).shift(-1)  # we want them in pips
df_trades['GAIN']= df_trades['DELTA'] * df_trades['IS_TRADE'] # calculate the actual gain in our trades, negative means to sell

In [11]:
df_trades['GAIN'].sum()

798.0000000000018

In [10]:
from dateutil.parser import *
df_trades['time'] = [parse(x) for x in df_trades.time]

In [17]:
df_trades['DURATION']= df_trades.time.diff().shift(-1)
df_trades['DURATION']= [x.total_seconds() / 3600 for x in df_trades.DURATION]

In [18]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_9,MA_21,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,DURATION
7,2023-01-18 17:00:00+00:00,158.878,159.164,158.802,159.034,159.284333,159.354714,-0.070381,0.113302,-1,-27.0,27.0,22.0
29,2023-01-19 15:00:00+00:00,158.552,158.815,158.500,158.764,158.489000,158.472952,0.016048,-0.102921,1,232.1,232.1,112.0
93,2023-01-24 07:00:00+00:00,161.302,161.333,160.945,161.085,161.361556,161.427333,-0.065778,0.010921,-1,-46.9,46.9,19.0
112,2023-01-25 02:00:00+00:00,160.574,160.734,160.456,160.616,160.556778,160.549095,0.007683,-0.059603,1,-101.0,-101.0,9.0
121,2023-01-25 11:00:00+00:00,159.904,159.926,159.512,159.606,160.399222,160.445810,-0.046587,0.010349,-1,107.8,-107.8,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4939,2023-11-02 04:00:00+00:00,183.314,183.339,183.184,183.204,183.320333,183.316524,0.003810,-0.022270,1,-2.6,-2.6,2.0
4941,2023-11-02 06:00:00+00:00,183.307,183.310,183.105,183.178,183.280111,183.281286,-0.001175,0.006429,-1,34.3,-34.3,11.0
4952,2023-11-02 17:00:00+00:00,183.337,183.528,183.296,183.521,183.274000,183.255571,0.018429,-0.040206,1,-10.9,-10.9,14.0
4966,2023-11-03 07:00:00+00:00,183.450,183.533,183.354,183.412,183.418222,183.419000,-0.000778,0.018302,-1,18.2,-18.2,2.0


In [87]:
df_plot = df_ma.iloc[0:50]

In [88]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))


fig.update_layout(width=1000,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()
